In [216]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
from scipy.stats import entropy
from scipy.optimize import minimize
from queue import PriorityQueue

In [217]:
class Packet:
    def __init__(self,index,time,src,dst,pre,cur,type):
        self.index = index
        self.t = time
        self.src = src
        self.dst = dst
        self.pre = pre
        self.cur = cur
        self.next = cur
        self.type = type 

In [218]:
class Link:
    def __init__(self,index,start,end,success_rate,type = 'generic link',mean_success_rate = 0.01,success_num = 0):
        self.index = index
        self.start = start
        self.end = end
        self._success_rate = success_rate
        self._success_num = success_num
        self.type = type
        self.mean_success_rate = mean_success_rate
        self.attempt_num = 0
        self.ETC = float('inf')
        self.path_cost = {}
        self.t = 1
        
    def __str__(self):
        return f"Link type: {self.type},  start: {self.node1},  end: {self.node2}"
    def Jt(self,end_node):
        if end_node in self.path_cost.keys():
            return self.path_cost[end_node]
        else:
            self.path_cost[end_node] = float('inf')
            return self.path_cost[end_node]
        
    
    def transmit(self,end_node,time):
        result = (np.random.random() <= self._success_rate)
        self.attempt_num += 1
        if result is True:
            self._success_num += 1
        self.mean_success_rate = self._success_num/self.attempt_num
        self.path_cost[end_node] = 1
        return result


In [219]:
class Node:
    def __init__(self,index,dest):
        self.index = index
        self.dest = dest
        self.neighbor = set()
        self.link = {}
        self.cost = {}
        self.neighbor_cost = {}
        self.t = 1
        
    def __str__(self):
        return f"Node {self.index}"
    def add_neighbor(self,link,neighbor_index):
        if neighbor_index in self.neighbor:
            self.link[neighbor_index] = link
        else:
            self.neighbor.add(neighbor_index)
            self.link[neighbor_index] = link
            self.neighbor_cost[neighbor_index] = float('inf')
    def del_neighbor(self,neighbor_index):
        if neighbor_index in self.neighbor:
            self.neighbor.discard(neighbor_index)
            self.link.pop(neighbor_index)
            self.neighbor_cost.pop(neighbor_index)                   


In [220]:
def kl_divergence(p, q):
    return p * math.log(p / q) + (1 - p) * math.log((1 - p) / (1 - q))

In [221]:
class Policy:
    def __str__(self):
        return 'generic policy'
    def choose(self):
        return 0

class Totoro(Policy):
    def __init__(self,node,end_node,time,C):
        self.node = node
        self.end_node = end_node
        self.t = time
        self.C = C
    def Update_ETC(self):
        for node,link in self.node.link.items():
            def objective(u):
                if u == 0: return float('inf')
                else: return 1/u
            def constraint(u):
                return link.attempt_num * kl_divergence(link.mean_success_rate, u) - self.C * math.log(self.t)
            cons = {'type': 'ineq', 'fun': constraint}
            result = minimize(objective,x0= link.mean_success_rate,bounds=[(link.mean_success_rate, 1)], constraints=cons,method='SLSQP')
            if result.success is True:
                link.ETC = result.fun
            else:
                link.ETC  = float('inf')
    def choose(self):
        self.Update_ETC()
        cost = {}
        min_value = float('inf')
        min_index = []
        for node,link in self.node.link.items():
            cost[link.end] = link.ETC + link.Jt(self.end_node)
            print(f"link{link.index}, ETC: {link.ETC}, Jt: {link.Jt(self.end_node)}")
            if(cost[link.end] == min_value):
                min_index.append(link.index)
                min_value = cost[link.end]
            elif (cost[link.end] < min_value):
                min_index = [link.index]
                min_value = cost[link.end]
        print(min_index)
        final_index = random.choice(min_index)
        return final_index
            
        
        

In [222]:
n = {}
n[1] = Node(1,4)
n[2] = Node(2,4)
n[3] = Node(3,4)
n[4] = Node(4,4)
l = {}
l[1] = Link(1,1,2,0.5)
n[1].add_neighbor(l[1],n[2])
l[2] = Link(2,1,3,0.5)
n[1].add_neighbor(l[2],n[3])
l[3] = Link(3,2,4,0.9)
n[2].add_neighbor(l[3],n[4])
l[4] = Link(4,3,4,0.1)
n[3].add_neighbor(l[4],n[4])
const = 0.8
total_attempt_num = 0
p = {}
event = PriorityQueue()
for i in range(1,500):
    p[i] = Packet(i,i*5,1,4,1,1,1)
    event.put((i*5,i))
while event.empty() is False:
    packet = p[event.get()[1]]
    n[packet.cur].t = packet.t
    print(f"packet{packet.index}, curID{packet.cur}")
    if (packet.cur == packet.dst):
        print(f"packet{packet.index} arrived")
        continue
    total_attempt_num += 1
    packet.t += 1
    cur_node = n[packet.cur]
    best_link= Totoro(cur_node, packet.dst, packet.t,const).choose()
    result = l[best_link].transmit(packet.dst,packet.t)
    if result is True:
        packet.next = l[best_link].end
        packet.cur = l[best_link].end
    else:
        print(f"packet{packet.index} transmit fail at link{best_link}")
        packet.next = packet.src
        packet.cur = packet.src
    event.put((packet.t,packet.index))
    print(f"link{best_link} mean success rate:{l[best_link].mean_success_rate} attempt num:{l[best_link].attempt_num}")
print(f"Total Attempt Num: {total_attempt_num}")
    

packet1, curID1
link1, ETC: inf, Jt: inf
link2, ETC: inf, Jt: inf
[1, 2]
packet1 transmit fail at link2
link2 mean success rate:0.0 attempt num:1
packet1, curID1


/var/folders/92/v1h0bcpx2ll_yj2p51s5lpz80000gn/T/ipykernel_945/1252363589.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return p * math.log(p / q) + (1 - p) * math.log((1 - p) / (1 - q))
/var/folders/92/v1h0bcpx2ll_yj2p51s5lpz80000gn/T/ipykernel_945/1252363589.py:2: RuntimeWarning: divide by zero encountered in divide
  return p * math.log(p / q) + (1 - p) * math.log((1 - p) / (1 - q))
/var/folders/92/v1h0bcpx2ll_yj2p51s5lpz80000gn/T/ipykernel_945/1252363589.py:2: RuntimeWarning: invalid value encountered in divide
  return p * math.log(p / q) + (1 - p) * math.log((1 - p) / (1 - q))


ValueError: math domain error